# 전세 거래 건수 통한 전세 피해 예측

- 부동산 유형 (`property_type`): 부동산 유형 (아파트, 오피스텔, 빌라)
- 보증금 액수 (`deposit_amount`): 보증금 금액 (억 단위)
- 보증금의 비율 (`deposit_ratio`): 보증금의 비율 (0~1 사이의 소수점)
- 소유자 연령대 (`owner_age`): 소유자의 나이 (20~79세)
- 소유자 채무 잔액 (`owner_debt`): 소유자의 채무 잔액 (억 단위)
- 임차인 신뢰 상태 (`tenant_trust`): 임차인의 신뢰 상태 (신뢰 가능, 주의 요망, 높은 위험)
- 계약 만료 여부 (`contract_expiry`): 계약 만료 여부 (True: 만료, False: 미만료)
- 결혼 상태 (`marriage_status`): 임차인의 결혼 상태 (청년, 신혼부부, 혼인10년 이상)
- 권리 침해 기록 (`legal_dispute`): 가압류, 가처분, 압류 등 권리 침해 기록 여부 (True: 있음, False: 없음)
- 사기 여부 (`fraudulent`): 해당 거래의 사기 여부 (True: 사기, False: 정상)

데이터 원본: https://www.kaggle.com/datasets/ranjeetshrivastav/fraud-detection-dataset

In [15]:
# 1. 패키지 불러오기 및 데이터 로드
import pandas as pd
import sweetviz as sv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
data = pd.read_excel('5조_데이터.xlsx')

In [ ]:
# 2. 데이터 구조 보기
print("데이터 구조:")
print(data.info())
print("\n데이터 샘플:")
print(data.head())

In [ ]:
# 3. Sweetviz를 활용한 EDA
eda_report = sv.analyze(data)
eda_report.show_notebook()

In [17]:
# 4. 데이터 분리
X = data.drop(columns=['fraudulent'])  # 독립 변수
y = data['fraudulent']                # 종속 변수
# 수치형 변수만 선택 (Random Forest는 숫자 데이터를 요구)
X = pd.get_dummies(X, columns=['property_type', 'tenant_trust', 'marriage_status'], drop_first=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [18]:
# 5. Random Forest 모델링
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

In [ ]:
# 6. 결과 평가
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, Accuracy: {accuracy:.2f}")

In [ ]:
# 7. Feature Importance 시각화
importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
sns.barplot(x=importances, y=features)
plt.title('Feature Importance')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()